<a href="https://colab.research.google.com/github/ARPM01/FasterRCNN_finetuned/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install matplotlib
!pip install numpy
!pip install torch
!pip install tqdm
!pip install torchvision
!pip install wandb

     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 212 kB 44.4 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 136 kB 48.6 MB/s 
     |████████████████████████████████| 1.1 MB 42.8 MB/s 
     |████████████████████████████████| 127 kB 45.4 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 144 kB 48.6 MB/s 
     |████████████████████████████████| 271 kB 48.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 1.8 MB 5.0 MB/s 


In [5]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   train.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	__pycache__/
	drinks/
	model_weights.pth
	wandb/

no changes added to commit (use "git add" and/or "git commit -a")


In [6]:
!git add train.ipynb

In [10]:
!git commit -a -m  "Revised train.ipynb"

[main fe9ba29] Revised train.ipynb
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite train.ipynb (98%)


In [5]:
!git push gdrive master

fatal: not a git repository (or any of the parent directories): .git


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/Colab Notebooks/FasterRCNN_finetuned

/content/drive/MyDrive/Colab Notebooks/FasterRCNN_finetuned


In [4]:
ls

coco_eval.py   drinks/         model_weights.pth  train.ipynb    wandb/
coco_utils.py  engine.py       __pycache__/       transforms.py
config.py      label_utils.py  README.md          utils.py


In [15]:
import torch
import numpy as np
import wandb
import torchvision
import os

import label_utils
import utils

from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image

from engine import train_one_epoch, evaluate
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [31]:
wandb.login()
config = {
    "num_workers": 0,
    "pin_memory": False,
    "batch_size": 8,
    "dataset": "drinks",
    "train_split": "drinks/labels_train.csv",
    "test_split": "drinks/labels_test.csv",}
run = wandb.init(project="197_Assignment_2", entity="arpm", config=config)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [32]:
test_dict, test_classes = label_utils.build_label_dictionary(
    config['test_split'])
train_dict, train_classes = label_utils.build_label_dictionary(
    config['train_split'])

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, root, dictionary, transforms=None):
        self.dictionary = dictionary
        self.root = root
        self.transforms = transforms
        self.imgs = list(os.listdir(os.path.join(root, "drinks")))
    def __len__(self):
        return len(self.dictionary)

    def __getitem__(self, idx):

        key = list(self.dictionary.keys())[idx]
        img = Image.open(key)
        
        num_objs = len(self.dictionary[key])

        b_temp = self.dictionary[key]
        b_temp2 = [[j[i] for i in range(4)] for j in b_temp]  #remove class from value
        boxes = [[i[0], i[2], i[1], i[3]] for i in b_temp2] #in [xmin, ymin, xmax, ymax]

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.tensor([i[4] for i in b_temp], dtype=torch.int64)
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area

        if self.transforms:
            img = self.transforms(img)

        return img, target

train_split = ImageDataset("", train_dict, transforms.ToTensor())
test_split = ImageDataset("", test_dict, transforms.ToTensor())

print("Train split len:", len(train_split), train_split)
print("Test split len:", len(test_split), test_split)

train_loader = DataLoader(train_split,
                          batch_size=config['batch_size'],
                          shuffle=False,
                          num_workers=config['num_workers'],
                          pin_memory=config['pin_memory'],
                          collate_fn=utils.collate_fn)

test_loader = DataLoader(test_split,
                         batch_size=config['batch_size'],
                         shuffle=False,
                         num_workers=config['num_workers'],
                         pin_memory=config['pin_memory'],
                         collate_fn=utils.collate_fn)

Train split len: 996 <__main__.ImageDataset object at 0x7f87bdc61f10>
Test split len: 51 <__main__.ImageDataset object at 0x7f87bdc61e90>


In [33]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
num_epochs = 10

for epoch in range(num_epochs):
  train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=5)
  lr_scheduler.step()
  evaluate(model, test_loader, device=device)

wandb.finish()
print("Saving the trained model")
torch.save(model.state_dict(), 'model_weights.pth')

Epoch: [0]  [  0/125]  eta: 0:12:46  lr: 0.000045  loss: 1.2773 (1.2773)  loss_classifier: 1.1261 (1.1261)  loss_box_reg: 0.1228 (0.1228)  loss_objectness: 0.0204 (0.0204)  loss_rpn_box_reg: 0.0079 (0.0079)  time: 6.1314  data: 0.1093  max mem: 10085
Epoch: [0]  [  5/125]  eta: 0:13:53  lr: 0.000247  loss: 1.2773 (1.2870)  loss_classifier: 1.1261 (1.1434)  loss_box_reg: 0.1220 (0.1220)  loss_objectness: 0.0148 (0.0164)  loss_rpn_box_reg: 0.0048 (0.0052)  time: 6.9426  data: 1.1621  max mem: 10408
Epoch: [0]  [ 10/125]  eta: 0:12:28  lr: 0.000448  loss: 1.1354 (1.0864)  loss_classifier: 1.0087 (0.9559)  loss_box_reg: 0.1090 (0.1091)  loss_objectness: 0.0148 (0.0162)  loss_rpn_box_reg: 0.0054 (0.0052)  time: 6.5051  data: 0.6821  max mem: 10408
Epoch: [0]  [ 15/125]  eta: 0:11:38  lr: 0.000650  loss: 0.8619 (0.8801)  loss_classifier: 0.7403 (0.7585)  loss_box_reg: 0.0995 (0.1016)  loss_objectness: 0.0144 (0.0151)  loss_rpn_box_reg: 0.0048 (0.0049)  time: 6.3475  data: 0.5008  max mem: 10

KeyError: ignored